##Master Window Function

Window functions:

* Lead() - To compare current row and next row
* Lag()  - To compare current row and previous row
* Row_number() - It assigns unique sequencial value to each rows.
* Dense_rank() - It assigns unique value to rows but if its repeated value assigns same rank and not gives any gap to the next row like rank function.

* Rank() - It assigns rank to rows but skips numbers for duplicates.


Key Functions:

* `Sum() , Over(), Avg(), Over()` - Rolling aggregates without `Group By`

###Find the Previous Order date for each customer

In [0]:
%sql
select customer_id,order_date,
  lag() over(partition by customer_id order by order_date) AS Prev_Order
from orders;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2887099974568108>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2887099974568108>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("c2VsZWN0IGN1c3RvbWVyX2lkLG9yZGVyX2RhdGUsCiAgbGFnKCkgb3ZlcihwYXJ0aXRpb24gYnkgY3VzdG9tZXJfaWQgb3JkZXIgYnkgb3JkZXJfZGF0ZSkgQVMgUHJldl9PcmRlcgpmcm9tIG9yZGVycw==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_n

###Use Common Table Expression (CTEs) Instead of subqueries:

CTEs (`WITH` clause) make queries more readable, modular, and optimized.

Regular CTE (Non-Recursive CTE)

* Works like a temporary view - computed once and then used.
* Used for modularizing complex queries into smaller, readable parts.



###Find the customers who placed High-Value Orders

In [0]:
With HighValueOrders As (
    select customer_id, Sum(amount) as Total_spent
    from orders
    group by customer_id
    having sum(amount)>10000
)
select * from HighValueOrders;

###How to Determine a "High-Value Order" Threshold?

Use Percentile Analysis

In [0]:
PERCENTILE_CONT()


###Handle NULLs 

* `NULL` values behave unexpectedly in filters and aggregates.
* Use `COALESCE()` to replace `NULL` values with defaults.
* Always use `IS NULL` instead of `=NULL` in conditions.

In [0]:
%sql
CREATE TABLE Employees (
    EmployeeID INT,
    EmployeeName STRING,
    Department STRING
);


In [0]:
%sql
INSERT INTO Employees (EmployeeID, EmployeeName, Department) VALUES
(1, 'John Doe', 'HR'),
(2, 'Jane Smith', 'Finance'),
(3, 'John Doe', 'HR'),
(4, 'Jane Smith', 'Finance');


In [0]:
%sql
select * from Employees;

###Cumulative Sum Query

In [0]:
%sql
CREATE TABLE transactions (
    customer_id INT,
    transaction_date DATE,
    amount DECIMAL(10,2)
);

INSERT INTO transactions (customer_id, transaction_date, amount) VALUES
(1, '2023-01-01', 100.00),
(1, '2023-02-01', 200.00),
(2, '2023-01-01', 250.00),
(2, '2023-02-01', 250.00),
(3, '2023-01-01', 300.00);


In [0]:
%sql
select * from  transactions;

In [0]:
%sql

select customer_id,
transaction_date,
 sum(amount) over(partition by customer_id order by transaction_date asc) as amt
from transactions


###COMPLEX SQL Joins with 3 tables

In [0]:
%fs ls dbfs:/user/hive/warehouse/customers

In [0]:
%sql
DROP Table if EXISTS customers;

In [0]:
%sql
CREATE TABLE Customers (
    customer_id INT,
    name STRING
)
USING DELTA;


In [0]:
%sql
CREATE TABLE Orders (
    order_id INT,
    customer_id INT,
    order_date DATE
)
USING DELTA;


In [0]:
%sql
CREATE TABLE Payments (
    payment_id INT,
    order_id INT,
    payment_date DATE,
    amount DECIMAL(10,2)
)
USING DELTA;


In [0]:
%sql
-- Insert Customers
INSERT INTO Customers (customer_id, name) VALUES 
(1, 'Alice'),
(2, 'Bob'),
(3, 'Charlie'),
(4, 'David');

-- Insert Orders
INSERT INTO Orders (order_id, customer_id, order_date) VALUES 
(101, 1, '2024-01-15'),
(102, 2, '2024-02-01'),
(103, 3, '2024-02-10'),
(104, 4, '2024-02-15'),
(105, 1, '2024-02-20'); -- Alice made another order

-- Insert Payments (Only some customers made payments)
INSERT INTO Payments (payment_id, order_id, payment_date, amount) VALUES 
(201, 101, '2024-01-16', 150.00), -- Alice paid for order 101
(202, 102, '2024-02-02', 200.00), -- Bob paid for order 102
(203, 104, '2024-02-20', 100.00); -- David paid for order 104
-- Charlie (order_id 103) and Alice (order_id 105) have not paid


In [0]:
%sql
select * from Customers;
select * from Orders ;
select * from Payments;

##Query to Find Customers Who Made Orders but Have Not Paid in the Last 30 Days



In [0]:
%sql
CURRENT_DATE()

In [0]:
%sql
SELECT C.CUSTOMER_ID, C.NAME
from CUSTOMERS C
join ORDERS O 
ON C.CUSTOMER_ID = O.CUSTOMER_ID
LEFT JOIN PAYMENTS P 
ON O.ORDER_ID = P.ORDER_ID
AND P.payment_date>NOW() - INTERVAL 30 DAY
Where p.order_id is NULL; 

In [0]:
%sql
CREATE TABLE Service_Requests (
    request_id INT,
    created_at TIMESTAMP,
    resolved_at TIMESTAMP
) USING DELTA;


In [0]:
%sql
drop table Employees;

In [0]:
%sql
CREATE TABLE Employe (
    emp_id INT,
    emp_name STRING,
    department STRING,
    salary DOUBLE
) USING DELTA;


In [0]:
%sql
show tables

In [0]:
%sql
INSERT into employe VALUES 
    (1, 'Alice', 'IT', 70000),
    (2, 'Bob', 'IT', 60000),
    (3, 'Charlie', 'IT', 80000),
    (4, 'David', 'Finance', 90000),
    (5, 'Eve', 'Finance', 85000),
    (6, 'Frank', 'Finance', 95000),
    (7, 'Grace', 'HR', 50000),
    (8, 'Hank', 'HR', 55000),
    (9, 'Ivy', 'HR', 45000);



###Percentile Approx()

In [0]:
%sql
select department,
  percentile_approx(SALARY,0.5) AS MEDIAN_SALARY
from employe
GROUP BY department;

In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/assessments")

dbutils.fs.rm("dbfs:/user/hive/warehouse/assessments", recurse=True)


In [0]:
%sql
DROP TABLE IF EXISTS assessments;



In [0]:
%sql
CREATE TABLE assessments (
    id INT,
    experience INT NOT NULL,
    sql INT,
    algo INT,
    bug_fixing INT
) USING DELTA
LOCATION 'dbfs:/mnt/external_storage/assessments';



In [0]:
%sql
insert into assessments values 
(1,2,null,null,null),
(2,20,null,null,20),
(3,7,100,null,100),
(4,3,100,50,null),
(5,2,40,100,100);

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql

select * from assessments;

id,experience,sql,algo,bug_fixing
1,2,null,null,null
2,20,null,null,20
3,7,100,null,100
4,3,100,50,null
5,2,40,100,100


In [0]:
%sql 
select experience as exp,
    SUM(case
         when (
         (CASE WHEN sql IS NULL or sql=100 then 1 else 0 end ) +
          (CASE WHEN  algo IS NULL or algo=100 then 1 else 0 end ) +
          (CASE WHEN  bug_fixing IS NULL or bug_fixing=100 then 1 else 0 end)
        ) =3
        THEN 1 else 0 END ) AS max,
        count(*) as count
from assessments
group by experience
order by experience DESC;


exp,max,count
20,0,1
7,1,1
3,0,1
2,1,2


In [0]:
%sql
select * from assessments;

id,experience,sql,algo,bug_fixing
1,2,null,null,null
2,20,null,null,20
3,7,100,null,100
4,3,100,50,null
5,2,40,100,100


In [0]:
%sql
select id, experience,
  case when sql IS NULL or sql= 100 then 1 else 0 end as Perfect_sql_score ,
  case when algo is NULL or algo=100 then 1 else 0 end as Perfect_algo_score,
  case when bug_fixing is NULL or bug_fixing=100 then 1 else 0 end as Perfect_bug_fixing_score
from assessments;

In [0]:
%sql
SELECT experience,
  SUM(CASE WHEN (case WHEN sql IS NULL OR sql = 100 THEN 1 ELSE 0 END +
  CASE WHEN algo IS NULL OR algo = 100 THEN 1 ELSE 0 END +
  CASE WHEN bug_fixing IS NULL OR bug_fixing = 100 THEN 1 ELSE 0 END)=3 then 1 else 0 end) AS max_score_flag,
count(*) as Total_candidates
FROM assessments
GROUP BY experience


In [0]:
%sql
SELECT experience,
  SUM(
    CASE 
      WHEN ( 
        (CASE WHEN sql IS NULL OR sql = 100 THEN 1 ELSE 0 END) + 
        (CASE WHEN algo IS NULL OR algo = 100 THEN 1 ELSE 0 END) + 
        (CASE WHEN bug_fixing IS NULL OR bug_fixing = 100 THEN 1 ELSE 0 END) 
      ) = 3 
      THEN 1 ELSE 0 
    END
  ) AS max_score_flag,
  COUNT(*) AS Total_candidates
FROM assessments
GROUP BY experience
ORDER BY experience DESC;



In [0]:
%sql
select * from assessments;

In [0]:
%sql
CREATE TABLE assessments2 (
    id INT,
    experience INT,
    sql INT,
    algo INT,
    bug_fixing INT
) USING DELTA;


In [0]:
%sql
select * from assessments;

In [0]:
%sql
INSERT INTO assessments2 (
 VALUES 
(1, 2, NULL, NULL, NULL),
(2, 20, NULL, NULL, 20),
(3, 7, 100, NULL, 100),
(4, 3, 100, 50, NULL),
(5, 2, 40, 100, 100);


In [0]:
%sql
select * from assessments;

In [0]:
%sql
SELECT experience AS exp,
       SUM((sql IS NULL OR sql = 100) * 
           (algo IS NULL OR algo = 100) * 
           (bug_fixing IS NULL OR bug_fixing = 100)) AS max,
       COUNT(*) AS count
FROM assessments
GROUP BY experience
ORDER BY experience DESC;


In [0]:
%sql
SELECT experience AS exp,
       SUM((sql IS NULL OR sql = 100) * 
           (algo IS NULL OR algo = 100) * 
           (bug_fixing IS NULL OR bug_fixing = 100)) AS max,
       COUNT(*) AS count
FROM assessments
GROUP BY experience
ORDER BY experience DESC;


In [0]:
%sql
create database uber;

In [0]:
%sql

create table rides (
  ride_id INT primary key,
  driver_id INT not null,
  rider_id INT,
  pickup_location VARCHAR(100),
  dropoff_location VARCHAR(100),
  ride_date DATETIME,
  fare DECIMAL(10,2) NOT NULL
);

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-2021183687507497>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2021183687507497>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("Y3JlYXRlIHRhYmxlIHJpZGVzICgKICByaWRlX2lkIElOVCAsCiAgZHJpdmVyX2lkIElOVCwKICByaWRlcl9pZCBJTlQsCiAgcGlja3VwIGxvY2F0aW9uIFZBUkNIQVIoMTAwKSwKICBkcm9wb2ZmX2xvY2F0aW9uIFZBUkNIQVIoMTAwKSwKICByaWRlX2RhdGUgREFURVRJTUUsCiAgZmFyZSBERUNJTUFMKDEwLDIpIE5PVCBOVUxMCgop").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:


--𝐏𝐫𝐨𝐛𝐥𝐞𝐦 - You have an Employees table where each employee can have a manager, and employees can report to other employees, including possibly themselves (self-reporting). You need to find all the managers who have at least 7 direct reports.


In [0]:
%sql
create database sql_query;

In [0]:
%sql
CREATE TABLE Employee(
    employee_id INT NOT NULL,
    employee_name STRING,
    manager_id INT
) USING DELTA;

ALTER TABLE Employee ADD CONSTRAINT employee_id_not_null CHECK (employee_id IS NOT NULL);


In [0]:
%sql

INSERT INTO Employee (employee_id, employee_name, manager_id) VALUES
(1, 'Alice', 3),
(2, 'Bob', 3),
(3, 'Charlie', 4),
(4, 'David', 5),
(5, 'Eve', 5),
(6, 'Frank', 5),
(7, 'Grace', 5),
(8, 'Hannah', 5),
(9, 'Ivan', 5),
(10, 'Jack', 5),
(11, 'Karen', 4),
(12, 'Leo', 4),
(13, 'Mike', 5),
(14, 'Nina', 5),
(15, 'Olivia', 5);


num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql

select * from Employee;

employee_id,employee_name,manager_id
1,Alice,3
2,Bob,3
3,Charlie,4
4,David,5
5,Eve,5
6,Frank,5
7,Grace,5
8,Hannah,5
9,Ivan,5
10,Jack,5


In [0]:
%sql
select m.employee_name as manager_name,
count(e.employee_id) as report_count
from Employee e join Employee m
on e.manager_id = m.employee_id
group by m.employee_name
having report_count>7

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2752570810307506>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2752570810307506>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("c2VsZWN0IG0uZW1wbG95ZWVfbmFtZSBhcyBtYW5hZ2VyX25hbWUsCmNvdW50KGUuZW1wbG95ZWVfaWQpIGFzIHJlcG9ydF9jb3VudApmcm9tIEVtcGxveWVlIGUgam9pbiBFbXBsb3llZSBtCm9uIGUubWFuYWdlcl9pZCA9IG0uZW1wbG95ZWVfaWQKZ3JvdXAgYnkgbS5lbXBsb3llZV9uYW1lCmhhdmluZyByZXBvcnRfY291bnQ+Nw==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:


-- aggregation function , group by , having , alaises 

In [0]:
%sql

create database test;

In [0]:
%sql
use test

In [0]:
%sql
DROP table customerTransactions;


In [0]:
%sql
CREATE TABLE CustomerTransactions (
    id INT,
    login_device STRING,
    customer_name STRING,
    ip_address STRING,
    product STRING,
    amount DECIMAL(10,2),
    is_placed BOOLEAN,
    is_viewed BOOLEAN,
    transaction_status STRING
) USING DELTA;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2923474654458748>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2923474654458748>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("Q1JFQVRFIFRBQkxFIEN1c3RvbWVyVHJhbnNhY3Rpb25zICgKICAgIGlkIElOVCwKICAgIGxvZ2luX2RldmljZSBTVFJJTkcsCiAgICBjdXN0b21lcl9uYW1lIFNUUklORywKICAgIGlwX2FkZHJlc3MgU1RSSU5HLAogICAgcHJvZHVjdCBTVFJJTkcsCiAgICBhbW91bnQgREVDSU1BTCgxMCwyKSwKICAgIGlzX3BsYWNlZCBCT09MRUFOLAogICAgaXNfdmlld2VkIEJPT0xFQU4sCiAgICB0cmFuc2FjdGlvbl9zdGF0dXMgU1RSSU5HCikgVVNJTkcgREVMVEE=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_

In [0]:
%sql
MERGE INTO CustomerTransactions AS target
USING (SELECT 101 AS id, 'Mobile' AS login_device, 'Alice' AS customer_name, '192.168.1.1' AS ip_address, 'Laptop' AS product, 1000.50 AS amount, TRUE AS is_placed, FALSE AS is_viewed, 'Completed' AS transaction_status) AS source
ON target.id = source.id
WHEN MATCHED THEN 
  UPDATE SET target.transaction_status = source.transaction_status
WHEN NOT MATCHED THEN 
  INSERT *;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
%sql

INSERT INTO CustomerTransactions VALUES
(1, 'Mobile', 'Ravi', '192.168.1.1', 'Laptop', 50000.00, TRUE, FALSE, 'Completed'),
(2, 'Desktop', 'Priya', '192.168.1.2', 'Smartphone', 20000.00, TRUE, TRUE, 'Completed'),
(3, 'Tablet', 'Arjun', '192.168.1.3', 'Headphones', 1500.00, FALSE, TRUE, 'Failed'),
(4, 'Mobile', 'Meena', '192.168.1.4', 'Shoes', 2500.00, TRUE, FALSE, 'Completed'),
(5, 'Desktop', 'Karthik', '192.168.1.5', 'Watch', 5000.00, TRUE, TRUE, 'Completed'),
(6, 'Mobile', 'Sowmya', '192.168.1.6', 'Tablet', 15000.00, TRUE, TRUE, 'Completed'),
(7, 'Tablet', 'Ramesh', '192.168.1.7', 'Smartphone', 25000.00, FALSE, TRUE, 'Failed'),
(8, 'Desktop', 'Divya', '192.168.1.8', 'Laptop', 60000.00, TRUE, FALSE, 'Completed'),
(9, 'Mobile', 'Arun', '192.168.1.9', 'Smartwatch', 12000.00, TRUE, TRUE, 'Completed'),
(10, 'Tablet', 'Deepa', '192.168.1.10', 'Laptop', 55000.00, FALSE, FALSE, 'Pending');


num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
select * from CustomerTransactions
where transaction_status in ('Failed','pending')

id,login_device,customer_name,ip_address,product,amount,is_placed,is_viewed,transaction_status
3,Tablet,Arjun,192.168.1.3,Headphones,1500.00,false,true,Failed
7,Tablet,Ramesh,192.168.1.7,Smartphone,25000.00,false,true,Failed


In [0]:
%sql
select count(id) from CustomerTransactions
where transaction_status='Failed'

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2923474654458751>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2923474654458751>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("c2VsZWN0IGNvdW50KGlkKSBmcm9tIEN1c3RvbWVyVHJhbnNhY3Rpb25zCndoZXJlIHRyYW5zYWN0aW9uX3N0YXR1cz0nRmFpbGVkJw==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() 

In [0]:
%sql

select 
case
    when 9<20 then 'True'
    else 'False'
end as test


test
True


In [0]:
%sql
drop table /user/hive/warehouse/customerdata

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-3101931010166563>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-3101931010166563>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("ZHJvcCB0YWJsZSBDdXN0b21lckRhdGE=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databrick

In [0]:
%sql
Create database cusdata

In [0]:
%sql
use cusdata

In [0]:
%sql
-- Case When Statement

CREATE TABLE CustomerData (
    id INT NOT NULL,
    customer_name VARCHAR(100),
    email VARCHAR(100),
    phone_number VARCHAR(15),
    address VARCHAR(200),
    amount DECIMAL(10, 2)
);

INSERT INTO CustomerData VALUES
(1, 'Ravi', 'ravi@example.com', '98765', 'Chennai', 5000.00),
(2, 'Priya', NULL, '98765', 'Bangalore', NULL),
(3, 'Arjun', 'arjun@example.com', NULL, 'Hyderabad', 1500.00),
(4, 'Meena', NULL, NULL, 'Mumbai', 2500.00),
(5, 'Karthik', 'karthik@example.com', '98765', NULL, 3000.00);


num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql

select * from customerData

id,customer_name,email,phone_number,address,amount
1,Ravi,ravi@example.com,98765,Chennai,5000.00
2,Priya,null,98765,Bangalore,null
3,Arjun,arjun@example.com,null,Hyderabad,1500.00
4,Meena,null,null,Mumbai,2500.00
5,Karthik,karthik@example.com,98765,null,3000.00


Find the person who spends high, low, and medium based on amount 

In [0]:
%sql
select 
    customer_name,
    amount,
    case 
        when amount>=4000 then 'High-Spender'
        when amount between 2000 and 4000 then 'Medium-spender'
        when amount<=2000 then 'Low-Spender'
    else 'no data'
    end as Spending
from customerData

customer_name,amount,Spending
Ravi,5000.00,High-Spender
Priya,null,no data
Arjun,1500.00,Low-Spender
Meena,2500.00,Medium-spender
Karthik,3000.00,Medium-spender


In [0]:
%sql
select * from customerData;

id,customer_name,email,phone_number,address,amount
1,Ravi,ravi@example.com,98765,Chennai,5000.00
2,Priya,null,98765,Bangalore,null
3,Arjun,arjun@example.com,null,Hyderabad,1500.00
4,Meena,null,null,Mumbai,2500.00
5,Karthik,karthik@example.com,98765,null,3000.00


####Write a condition who don't have phone number

In [0]:
%sql
select
    customer_name,
    case
        when email is null and phone_number is null then '9999'
        when phone_number is null then email
    else phone_number
    end as contact
    
from customerData


customer_name,contact
Ravi,98765
Priya,98765
Arjun,arjun@example.com
Meena,9999
Karthik,98765


In [0]:
%sql
SHOW DATABASES;


databaseName
cusdata
default


In [0]:
%sql
use cusdata

In [0]:
%sql

INSERT INTO employees (emp_id, emp_name, department, salary) VALUES
(1, 'Alice', 'HR', 55000),
(2, 'Bob', 'IT', 85000),
(3, 'Charlie', 'Finance', 65000),
(4, 'David', 'IT', 120000),
(5, 'Eve', 'HR', 45000),
(6, 'Frank', 'Finance', 95000);


num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql
select * from employees;

emp_id,emp_name,department,salary
1,Alice,HR,55000.0
2,Bob,IT,85000.0
3,Charlie,Finance,65000.0
4,David,IT,120000.0
5,Eve,HR,45000.0
6,Frank,Finance,95000.0


####Using CASE WHEN in an UPDATE Statement

In [0]:
%sql
ALTER TABLE employees ADD COLUMN bonus DECIMAL(10,2);


#### Update bonus Based on Salary Ranges

In [0]:
%sql
update employees
set bonus = case
    when salary<50000 then 5000
    when salary between 50000 and 90000 then 7000
    when salary>90000 then 9000
  else 0
  end ;


num_affected_rows
6


In [0]:
%sql
select * from employees;

emp_id,emp_name,department,salary,bonus
1,Alice,HR,55000.0,7000.00
2,Bob,IT,85000.0,7000.00
3,Charlie,Finance,65000.0,7000.00
4,David,IT,120000.0,9000.00
5,Eve,HR,45000.0,5000.00
6,Frank,Finance,95000.0,9000.00


###let's categorize employees based on their salary into Low,  Medium, and High salary grades.

In [0]:
%sql

select emp_name,
  salary,
  case 
      when salary <50000 then 'Low_Salary'
      when salary between 50000 and 90000 then 'Medium_Salary'
      when salary >90000 then 'High_salary'
      else 'grade'
  end as No_Data
from employees    


emp_name,salary,No_Data
Alice,55000.0,Medium_Salary
Bob,85000.0,Medium_Salary
Charlie,65000.0,Medium_Salary
David,120000.0,High_salary
Eve,45000.0,Low_Salary
Frank,95000.0,High_salary


In [0]:
%sql
SELECT * from customerdata;

id,customer_name,email,phone_number,address,amount
1,Ravi,ravi@example.com,98765,Chennai,5000.00
2,Priya,null,98765,Bangalore,null
3,Arjun,arjun@example.com,null,Hyderabad,1500.00
4,Meena,null,null,Mumbai,2500.00
5,Karthik,karthik@example.com,98765,null,3000.00


In [0]:
%sql
select customer_name,
email,
phone_number,
address
  from customerdata
where 
  phone_number IS NULL
  OR email is null
  OR address is null;

customer_name,email,phone_number,address
Priya,null,98765,Bangalore
Arjun,arjun@example.com,null,Hyderabad
Meena,null,null,Mumbai
Karthik,karthik@example.com,98765,null


In [0]:
%sql

select customer_name,
amount,coalesce(amount,null, null,99,null,00.00) as Using_coalesce,
IFNULL(amount,'0.00') as Using_ifnull
from Customerdata;

customer_name,amount,Using_coalesce,Using_ifnull
Ravi,5000.00,5000.00,5000.00
Priya,null,99.00,0.00
Arjun,1500.00,1500.00,1500.00
Meena,2500.00,2500.00,2500.00
Karthik,3000.00,3000.00,3000.00


####String Handling in SQL

In [0]:
%sql 
select * from customerdata

id,customer_name,email,phone_number,address,amount
1,Ravi,ravi@example.com,98765,Chennai,5000.00
2,Priya,null,98765,Bangalore,null
3,Arjun,arjun@example.com,null,Hyderabad,1500.00
4,Meena,null,null,Mumbai,2500.00
5,Karthik,karthik@example.com,98765,null,3000.00


In [0]:
%sql
CREATE TABLE CustomerDetail (
    CustomerID INT NOT NULL,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Email VARCHAR(100),
    PhoneNumber VARCHAR(20),
    Address VARCHAR(255)
);


In [0]:
%sql
INSERT INTO CustomerDetail (CustomerID, FirstName, LastName, Email, PhoneNumber, Address)
VALUES
(1, 'John', 'Doe', 'john.doe@example.com', '9876543210', '123 Main Street, NY'),
(2, 'Alice', 'Smith', 'alice.smith@example.com', '8765432109', '456 Park Avenue, CA'),
(3, 'Bob', 'Johnson', 'bob.johnson@example.com', '7654321098', '789 Broadway, TX'),
(4, 'Charlie', 'Brown', 'charlie.brown@example.com', '6543210987', '101 First Street, FL');


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql

select * from CustomerDetail;

CustomerID,FirstName,LastName,Email,PhoneNumber,Address
1,John,Doe,john.doe@example.com,9876543210,"123 Main Street, NY"
2,Alice,Smith,alice.smith@example.com,8765432109,"456 Park Avenue, CA"
3,Bob,Johnson,bob.johnson@example.com,7654321098,"789 Broadway, TX"
4,Charlie,Brown,charlie.brown@example.com,6543210987,"101 First Street, FL"


In [0]:
%sql
select len(FirstName) as name_length,
upper(Email) as Email_upper,
lower(Email) as lower_case,
concat(FirstName," ", LastName),
substring(Address, 1 ,10) as prefix_address,
Trim(' Charlie ') AS trimmed_name,
RPAD(Address, 30, '*') as left_padded_adress,
REPLACE(Address,' ','-') as replaced_space,
INSTR(FirstName, 'a') as position_of_A,
reverse(FirstName) as reversed_name
from CustomerDetail;

name_length,Email_upper,lower_case,"concat(FirstName, , LastName)",prefix_address,trimmed_name,left_padded_adress,replaced_Address,position_of_A,reversed_name
4,JOHN.DOE@EXAMPLE.COM,john.doe@example.com,John Doe,123 Main S,Charlie,"123 Main Street, NY***********","123-Main-Street,-NY",0,nhoJ
5,ALICE.SMITH@EXAMPLE.COM,alice.smith@example.com,Alice Smith,456 Park A,Charlie,"456 Park Avenue, CA***********","456-Park-Avenue,-CA",0,ecilA
3,BOB.JOHNSON@EXAMPLE.COM,bob.johnson@example.com,Bob Johnson,789 Broadw,Charlie,"789 Broadway, TX**************","789-Broadway,-TX",0,boB
7,CHARLIE.BROWN@EXAMPLE.COM,charlie.brown@example.com,Charlie Brown,101 First,Charlie,"101 First Street, FL**********","101-First-Street,-FL",3,eilrahC


In [0]:
%sql
select

###DateTime()

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS datetime_vs_timestamp;
USE default;


In [0]:
%sql
DROP TABLE IF EXISTS demo_events;

CREATE TABLE demo_events (
    event_id BIGINT GENERATED ALWAYS AS IDENTITY,
    event_name STRING,
    event_date TIMESTAMP,       -- Used like DATETIME in MySQL
    created_at TIMESTAMP DEFAULT current_timestamp()
)
USING DELTA;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1245628715805226>:8
      6     display(df)
      7     return df
----> 8   _sqldf = ____databricks_percent_sql()
      9 finally:
     10   del ____databricks_percent_sql

File <command-1245628715805226>:5, in ____databricks_percent_sql()
      3 import base64
      4 spark.sql(base64.standard_b64decode("RFJPUCBUQUJMRSBJRiBFWElTVFMgZGVtb19ldmVudHM=").decode())
----> 5 df = spark.sql(base64.standard_b64decode("Q1JFQVRFIFRBQkxFIGRlbW9fZXZlbnRzICgKICAgIGV2ZW50X2lkIEJJR0lOVCBHRU5FUkFURUQgQUxXQVlTIEFTIElERU5USVRZLAogICAgZXZlbnRfbmFtZSBTVFJJTkcsCiAgICBldmVudF9kYXRlIFRJTUVTVEFNUCwgICAgICAgLS0gVXNlZCBsaWtlIERBVEVUSU1FIGluIE15U1FMCiAgICBjcmVhdGVkX2F0IFRJTUVTVEFNUCBERUZBVUxUIGN1cnJlbnRfdGltZXN0YW1wKCkKKQpVU0lORyBERUxUQQ==").decode())
      6 display(df)
      7 return df

File /databricks/spark/python/pyspark/instrumentation_utils.

In [0]:
%sql
-- Enable support for DEFAULT column values
SET spark.databricks.sql.features.columnDefaults.enabled = true;


key,value
spark.databricks.sql.features.columnDefaults.enabled,true


In [0]:
%sql
DROP TABLE IF EXISTS demo_events;

CREATE TABLE demo_events (
    event_id BIGINT GENERATED ALWAYS AS IDENTITY,
    event_name STRING,
    event_date TIMESTAMP,
    created_at TIMESTAMP DEFAULT current_timestamp()
)
USING DELTA
TBLPROPERTIES (
  'delta.feature.allowColumnDefaults' = 'supported'
);


In [0]:
%sql
DROP TABLE IF EXISTS regex_samples;

CREATE TABLE regex_samples (
    id INT ,
    sample_text VARCHAR(100)
);


INSERT INTO regex_samples (id,sample_text) VALUES 
(1,'apple'),         -- id=1
(2,'Banana'),        -- id=2 (note the capital B)
(3,'cherry'),        -- id=3
(4,'date'),          -- id=4
(5,'elderberry'),    -- id=5
(6,'fig'),           -- id=6
(7,'grape'),         -- id=7
(8,'honeydew'),      -- id=8
(9,'running'),       -- id=9 (ends with "ing")
(10,'123abc');        -- id=10 (starts with digits)


num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql

select * from regex_samples

id,sample_text
1,apple
2,Banana
3,cherry
4,date
5,elderberry
6,fig
7,grape
8,honeydew
9,running
10,123abc


###Example 1: Match Strings That Start with “a”


In [0]:
%sql

SELECT * 
FROM regex_samples
WHERE sample_text REGEXP '^a';


id,sample_text
1,apple


In [0]:
%sql
SELECT * from regex_samples
where sample_text REGEXP '^[A-Za-z]+$';

id,sample_text
1,apple
2,Banana
3,cherry
4,date
5,elderberry
6,fig
7,grape
8,honeydew
9,running


In [0]:
%sql
select *
from regex_samples
where sample_text REGEXP 'ing$';

id,sample_text
9,running


In [0]:
%sql
SELECT * FROM regex_samples
WHERE sample_text REGEXP '^.{5}$';


id,sample_text
1,apple
7,grape


In [0]:
%sql
select *
from regex_samples
where sample_text REGEXP '^[0-9]{3}[A-Za-z]+$';

id,sample_text
10,123abc


##Connect SQL through Python

In [0]:
import mysql.connector
from mysql.connector import Error

try:
    #Establish the Connection
    connection = mysql.connector.connect (
        host = 'localhost',
        database = 'test',
        user='root'
        password='Root'
        
    )
    #Create cursor 